In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import glob
import os
import numpy as np
import h5py
import matplotlib.colors as colors
import random

import sys, os
# This is not super pretty, but I think this is the best way to import stuff from ../../../util?
CODE_ROOT = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
if CODE_ROOT not in sys.path:
    sys.path.insert(1, CODE_ROOT)

from util.sim_data_helpers import get_data_from_snap_folder, get_data_from_header, get_cosmo_parameters

In [ ]:
def density_hist(path, snapN, resolution, _range=None):
    
    # get the data
    coordinates = get_data_from_snap_folder(path, snapN, "PartType1", "Coordinates")
    
    box_size = get_data_from_header(path, snapN, 'BoxSize') # in kpc
    dm_particle_mass = get_data_from_header(path, snapN, 'MassTable')[1]*1e10  # in M_sun
    physical_bin_volume = ((box_size/1e3)/resolution)**2 * (box_size/1e3)  # this is a projection, so one axis has the full length of the box

    unit_density_of_bin = dm_particle_mass/physical_bin_volume
    
    if _range is None:
        _range = [[0, box_size], [0, box_size]]
    
    # calculate the hist
    h, xedges, yedges = np.histogram2d(coordinates[:, 0], coordinates[:, 1], bins=resolution, range=_range)

    h = h*unit_density_of_bin # convert to actual density (M_sun/Mpc)
    
    return h, xedges, yedges

In [ ]:
gp_numbers_to_use = [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9,
                     10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                     20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
                     30, 31, 32, 33,     35, 36, 37, 38, 39,
                     40, 41, 42, 43, 44,     46, 47,     49]  # number 34, 45, 48 only ran ultil z = 2
gp_indices = [i for i in range(len(gp_numbers_to_use))]
random.shuffle(gp_indices)  # randomize which boxes are shown
gp_indices = gp_indices[:20]

base_path = "/vera/u/jerbo/my_ptmp/L25n256_suite/"
snapN = 5

sort_param = []
for gp_index in gp_indices:
    gp = gp_numbers_to_use[gp_index]
    path = base_path + f"gridpoint{gp}/"

    Omega0, OmegaBaryon, OmegaLambda, HubbleParam = get_cosmo_parameters(path)
    sort_param.append((Omega0, gp_index))

_, sorted_indices = zip(*sorted(sort_param))

In [ ]:
sorted_indices

In [ ]:
fig, ax = plt.subplots(figsize=(20,20),dpi=500)

ax.set_xticks([])
ax.set_yticks([])
for spine in ax.spines.values():
    spine.set_visible(False)

gs = gridspec.GridSpec(5, 5, height_ratios=[1, 1, 1, 1, 0.05])
axes = []

norm_path = base_path + f"gridpoint{gp_numbers_to_use[sorted_indices[-1]]}/"

norm_h, xedges_ref, yedges_ref = density_hist(norm_path, snapN, 512)
norm_h = norm_h/norm_h.mean()

abs_max = max(abs(norm_h.min()), abs(norm_h.max()))

# vmin = norm_h[norm_h != 0].min()

norm = colors.LogNorm(vmin=1/norm_h.max(), vmax=norm_h.max())

for i, gp_index in enumerate(sorted_indices): 
    gp = gp_numbers_to_use[gp_index]
    path = base_path + f"gridpoint{gp}/"
    
    row = i//5
    col = i%5
    
    ax = fig.add_subplot(gs[row, col])

    h, xedges, yedges = density_hist(path, snapN, 512)
    
    h = h/h.mean()
    
    im = ax.imshow(h.T, origin='lower', cmap='vanimo', norm=norm,
                extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
    
    Omega0, OmegaBaryon, OmegaLambda, HubbleParam = get_cosmo_parameters(path)
    
    ax.set_title(r"$\Omega_m$="+f"{Omega0:.2f}"+r"; $\Omega_\Lambda$="+f"{OmegaLambda:.2f}"+r"; h="+f"{HubbleParam:.2f}")
    if col == 0:
        ax.set_ylabel("y [ckpc/h]")
    else:
        ax.tick_params('y', labelleft=False)
    
    ax.set_xlabel("x [ckpc/h]")
    ax.set_facecolor('black')
    axes.append(ax)

cbar_ax = fig.add_subplot(gs[4, :])
cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal')
cbar.set_label(r"$\delta$+1")
plt.tight_layout(rect=[0, 0.05, 1, 1])
plt.savefig("plots/overdensity_grid.pdf", format="PDF")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,20),dpi=500)

ax.set_xticks([])
ax.set_yticks([])
for spine in ax.spines.values():
    spine.set_visible(False)

gs = gridspec.GridSpec(5, 5, height_ratios=[1, 1, 1, 1, 0.05])
axes = []

norm_path = base_path + f"gridpoint{gp_numbers_to_use[sorted_indices[-1]]}/"

norm_h, xedges_ref, yedges_ref = density_hist(norm_path, snapN, 512)

vmin = norm_h[norm_h != 0].min()

norm = colors.LogNorm(vmin=vmin, vmax=norm_h.max())

for i, gp_index in enumerate(sorted_indices): 
    gp = gp_numbers_to_use[gp_index]
    path = base_path + f"gridpoint{gp}/"
    
    row = i//5
    col = i%5
    
    ax = fig.add_subplot(gs[row, col])

    h, xedges, yedges = density_hist(path, snapN, 512)
    
    im = ax.imshow(h.T, origin='lower', cmap='magma', norm=norm,
                extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
    
    Omega0, OmegaBaryon, OmegaLambda, HubbleParam = get_cosmo_parameters(path)
    
    ax.set_title(r"$\Omega_m$="+f"{Omega0:.2f}"+r"; $\Omega_\Lambda$="+f"{OmegaLambda:.2f}"+r"; h="+f"{HubbleParam:.2f}")
    if col == 0:
        ax.set_ylabel("y [ckpc/h]")
    else:
        ax.tick_params('y', labelleft=False)
    
    ax.set_xlabel("x [ckpc/h]")
    ax.set_facecolor('black')
    axes.append(ax)

cbar_ax = fig.add_subplot(gs[4, :])
cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal')
cbar.set_label(r"Density [$M_\odot$ / Mpc]")
plt.tight_layout(rect=[0, 0.05, 1, 1])
plt.savefig("plots/Density_grid.pdf", format="PDF")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,20),dpi=500)

ax.set_xticks([])
ax.set_yticks([])
for spine in ax.spines.values():
    spine.set_visible(False)

gs = gridspec.GridSpec(5, 5, height_ratios=[1, 1, 1, 1, 0.05])
axes = []

reference_path = "/vera/u/jerbo/my_ptmp/L25n256_suite/reference_point/"
norm_path = base_path + f"gridpoint{gp_numbers_to_use[sorted_indices[6]]}/"

reference_h, xedges_ref, yedges_ref  = density_hist(reference_path, snapN, 512)
reference_h = reference_h * 1e-9 # convert from M_sun/Mpc^3 to M_sun/kpc^3
norm_h, *_ = density_hist(norm_path, snapN, 512)
norm_h = norm_h * 1e-9 # convert from M_sun/Mpc^3 to M_sun/kpc^3
abs_max = max(abs(norm_h.min()), abs(norm_h.max()))
norm = colors.SymLogNorm(linthresh=1e2, vmin=-abs_max, vmax=abs_max, base=10)

for i, gp_index in enumerate(sorted_indices): 
    gp = gp_numbers_to_use[gp_index]
    path = base_path + f"gridpoint{gp}/"
    
    row = i//5
    col = i%5
    
    ax = fig.add_subplot(gs[row, col])

    h, xedges, yedges = density_hist(path, snapN, 512)

    h = h * 1e-9 # convert from M_sun/Mpc^3 to M_sun/kpc^3

    h = (h - reference_h) 
    
    im = ax.imshow(h.T, origin='lower', cmap='vanimo', norm=norm,
                extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
    
    Omega0, OmegaBaryon, OmegaLambda, HubbleParam = get_cosmo_parameters(path)
    
    ax.set_title(r"$\Omega_m$="+f"{Omega0:.2f}"+r"; $\Omega_\Lambda$="+f"{OmegaLambda:.2f}"+r"; h="+f"{HubbleParam:.2f}")
    if col == 0:
        ax.set_ylabel("y [ckpc/h]")
    else:
        ax.tick_params('y', labelleft=False)
    
    ax.set_xlabel("x [ckpc/h]")
    ax.set_facecolor('black')
    axes.append(ax)

cbar_ax = fig.add_subplot(gs[4, :])
cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal')
cbar.set_label(r"$\Delta$ Density [$M_\odot$/ckpc]")
plt.tight_layout(rect=[0, 0.05, 1, 1])
plt.savefig("plots/Delta_density_grid.pdf", format="PDF")
plt.show()